# Fault-Tolerant Binary Streams — Safety-Critical Logic

**SC-NeuroCore v3.7 — Stochastic Redundancy for Error Suppression**

This notebook demonstrates how encoding Boolean values as **redundant bitstreams**
(1024 bits per value) enables correct computation even under significant noise.

**Principle:** majority-vote decoding recovers the correct Boolean as long as
fewer than 50% of the bits are corrupted.

> CopyRight: (c) 1998-2026 Miroslav Sotek. All rights reserved.  
> License: GNU AFFERO GENERAL PUBLIC LICENSE v3 | Commercial Licensing Available  
> Contact: www.anulum.li   protoscience@anulum.li

In [ ]:
import random
from sc_neurocore_engine import BitStreamTensor

LENGTH = 1024   # redundancy: 1024 bits per Boolean
ERROR_RATE = 0.05  # 5% random bit-flip rate

print(f"SC-NeuroCore Fault-Tolerant Binary Streams Demo")
print(f"  Redundancy: {LENGTH} bits per Boolean")
print(f"  Error rate: {ERROR_RATE*100:.1f}%")

## Helper Functions

- **`make_constant`** — encode `True` as all-ones, `False` as all-zeros
- **`inject_errors`** — XOR with a sparse noise mask at controlled rate
- **`decode_majority`** — `True` if more than half the bits are set

In [ ]:
def make_constant(value: bool, length: int = 1024) -> BitStreamTensor:
    """Encode a Boolean as a constant bitstream."""
    if value:
        words = length // 64
        tail_bits = length % 64
        data = [0xFFFF_FFFF_FFFF_FFFF] * words
        if tail_bits > 0:
            data.append((1 << tail_bits) - 1)
        return BitStreamTensor.from_packed(data, length)
    else:
        words = (length + 63) // 64
        return BitStreamTensor.from_packed([0] * words, length)


def inject_errors(tensor: BitStreamTensor, error_rate: float, seed: int) -> BitStreamTensor:
    """XOR with a random noise stream to flip bits at the given rate."""
    rng = random.Random(seed)
    bits = [1 if rng.random() < error_rate else 0 for _ in range(tensor.length)]
    words = (tensor.length + 63) // 64
    data = [0] * words
    for i, b in enumerate(bits):
        if b:
            data[i // 64] |= 1 << (i % 64)
    noise = BitStreamTensor.from_packed(data, tensor.length)
    return tensor.xor(noise)


def decode_majority(tensor: BitStreamTensor) -> bool:
    """Majority-vote decoding."""
    return tensor.popcount() > tensor.length // 2


def stochastic_and(a, b):
    data = [wa & wb for wa, wb in zip(a.data, b.data)]
    return BitStreamTensor.from_packed(data, a.length)


def stochastic_or(a, b):
    data = [wa | wb for wa, wb in zip(a.data, b.data)]
    return BitStreamTensor.from_packed(data, a.length)


def stochastic_not(a):
    data = list(a.data)
    for i in range(len(data)):
        data[i] = ~data[i] & 0xFFFF_FFFF_FFFF_FFFF
    tail = a.length % 64
    if tail > 0 and data:
        data[-1] &= (1 << tail) - 1
    return BitStreamTensor.from_packed(data, a.length)

## Step 1: Encode and Inject Noise

Encode three Boolean values (`A=True, B=False, C=True`) and inject 5% random bit-flips.

In [ ]:
val_a, val_b, val_c = True, False, True

a = make_constant(val_a, LENGTH)
b = make_constant(val_b, LENGTH)
c = make_constant(val_c, LENGTH)

print(f"Encoded:  A={val_a} ({a.popcount()}/{LENGTH} ones)")
print(f"          B={val_b} ({b.popcount()}/{LENGTH} ones)")
print(f"          C={val_c} ({c.popcount()}/{LENGTH} ones)")

a_noisy = inject_errors(a, ERROR_RATE, seed=1)
b_noisy = inject_errors(b, ERROR_RATE, seed=2)
c_noisy = inject_errors(c, ERROR_RATE, seed=3)

print(f"\nAfter {ERROR_RATE*100:.1f}% noise:")
print(f"  A: {a_noisy.popcount()}/{LENGTH} ones -> decode={decode_majority(a_noisy)} (correct: {val_a})")
print(f"  B: {b_noisy.popcount()}/{LENGTH} ones -> decode={decode_majority(b_noisy)} (correct: {val_b})")
print(f"  C: {c_noisy.popcount()}/{LENGTH} ones -> decode={decode_majority(c_noisy)} (correct: {val_c})")

assert decode_majority(a_noisy) == val_a
assert decode_majority(b_noisy) == val_b
assert decode_majority(c_noisy) == val_c
print("\nAll decoded correctly despite noise!")

## Step 2: Boolean Logic on Noisy Streams

Bitwise AND, OR, NOT operate directly on the packed words.
Majority-vote decoding still recovers the correct Boolean result.

In [ ]:
# AND
and_result = stochastic_and(a_noisy, c_noisy)
print(f"A AND C: {decode_majority(and_result)} (expected {val_a and val_c}) [{and_result.popcount()}/{LENGTH}]")

# OR
or_result = stochastic_or(a_noisy, b_noisy)
print(f"A OR  B: {decode_majority(or_result)} (expected {val_a or val_b}) [{or_result.popcount()}/{LENGTH}]")

# NOT
not_result = stochastic_not(b_noisy)
print(f"NOT   B: {decode_majority(not_result)} (expected {not val_b}) [{not_result.popcount()}/{LENGTH}]")

# Complex: (A AND C) OR (NOT B)
complex_result = stochastic_or(stochastic_and(a_noisy, c_noisy), stochastic_not(b_noisy))
expected = (val_a and val_c) or (not val_b)
print(f"(A AND C) OR (NOT B): {decode_majority(complex_result)} (expected {expected})")

## Step 3: Error Tolerance Sweep

How high can the error rate go before majority-vote decoding fails?
With 1024-bit redundancy, we expect near-perfect decoding up to ~40% error rate.

In [ ]:
print("Error tolerance sweep (100 trials per rate):")
print(f"{'Rate':>8s}  {'Correct':>8s}")
print("-" * 20)

rates = [0.01, 0.05, 0.10, 0.15, 0.20, 0.30, 0.40, 0.45]
results = []

for rate in rates:
    successes = sum(
        1 for trial in range(100)
        if decode_majority(inject_errors(a, rate, seed=10000 + trial)) == val_a
    )
    pct = successes / 100 * 100
    results.append(pct)
    print(f"{rate*100:7.1f}%  {pct:7.1f}%")

print("\nStochastic redundancy provides robust error suppression!")

## Summary

| Error Rate | Decoding Accuracy |
|:---:|:---:|
| 1–20% | 100% |
| 30% | ~100% |
| 40% | ~100% |
| 45% | starts to degrade |

The theoretical threshold is 50% — below that, the majority vote always recovers
the correct value. With 1024-bit redundancy, even 40%+ error rates are tolerated
with high probability.

**Applications:** safety-critical embedded systems, radiation-hardened computing,
fault-tolerant FPGA logic.